In [6]:
import tensorflow as tf
import numpy as np
from pprint import pprint

## What is Tensorflow?
TensorFlow 使用數據流的方式根據各個操作之間的依賴關係來表示您的計算。間單來說 TensorFlow 是一種 low-level 的程式語言，並使用大量的矩陣運算。首先要建立一個 dataflow graph 的模型，接著透過 TensorFlow session 的建立來執行來執行 TensorFlow graph 模型。

## Tensorflow 的基本元素
TensorFlow 的基本元素由兩個獨立的部分所組成，分別為 `Graphs` 與 `Sessions`。

1. 建立 graph 模型 (使用 tf.Graph)
2. 執行 graph 模型 (使用 tf.Session)


## Why dataflow graphs?
![](https://www.tensorflow.org/images/tensors_flowing.gif)

dataflow 是一個常見的平行計算程式設計模型，在 dataflow graph 中每一個節點(node)代表每一個單元計算，並提供多種算術(Arithmetic)運算符號例如(tf.add、tf.multiply...等)。

## 建立 Graph
### tf.constant
constant 常數，在許多程式語言當中也有一樣的變數型態。他在宣告時就必須賦予初始值，建立之後就無法再修改值。

下面範例建立一個一維的常數內有兩筆浮點數，變數名稱設為 `const_a`，注意這個命名是給 Tensorflow 內部使用，建立完之後可以打印出來觀看詳細內容。若要印出變數內的數字必須經過建立一個 `session` 轉譯後再呼叫 `.run()` 函式執行 graph 的內容。最後程式結束時記得釋放掉剛建立好的 `session` 避免佔用資源。

![](./screenshot/img1.1-1.png)

In [14]:
# define graph
a = tf.constant([1., 2.], dtype=tf.float32, name='const_a')
print(a)

Tensor("const_a_2:0", shape=(2,), dtype=float32)


In [6]:
# create a session
sess = tf.Session()
print(sess.run(a))
sess.close()

[1. 2.]


### 算術運算符號(operations)
常見的運算符號分別有tf.add(加), tf.subtract(減), tf.multiply(乘), tf.divide(除)，下面範例新增兩個常數節點(x1與x2)並呼叫相加和相乘來做示範。

In [26]:
# define graph
x1 = tf.constant([1, 2], name='x1')
x2 = tf.constant([3, 4], name='x2')

y1 = tf.add(x1, x2)  # y1 = x1 + x2 
y2 = tf.multiply(x1, x2)  # y2 = x1 * x2 

print(y1)
print(y2)

Tensor("Add_9:0", shape=(2,), dtype=int32)
Tensor("Mul_12:0", shape=(2,), dtype=int32)


In [27]:
# create a session
with tf.Session() as sess:
    output1, output2 = sess.run([y1, y2])  # output1, output2 = sess.run(y1), sess.run(y2)
    print(output1)
    print(output2)

[4 6]
[3 8]


![](./screenshot/img1.1-2.png)

### tf.placeholder
這個函式有點類似C語言中的scanf()，簡單來說此變數儲存使用者所餵入的數值，誰也不知道這個數值為何等待直到收到數值。

In [5]:
# define graph
x = tf.placeholder(dtype=tf.float32, shape=[2, 2], name='Input')  # have to give the right shape
ones = tf.constant([[1, 1], [1, 1]], dtype=tf.float32, name='one')

result = x + ones
print(x)
print(result)

Tensor("Input_1:0", shape=(2, 2), dtype=float32)
Tensor("add_1:0", shape=(2, 2), dtype=float32)


In [7]:
# create a session
sess = tf.Session()
print(sess.run(result, feed_dict={x: [[1, -1], [0, 1]]}))
# print(sess.run(result, feed_dict={X: [0, -1]}))  # wrong shape!
sess.close()

[[2. 0.]
 [1. 2.]]


![](./screenshot/img1.1-3.png)

## tf.Variable
跟 constant 差別在於 Variable 變數可以隨意修改數值。

In [7]:
# define graph
x1 = tf.Variable(1, name='x1')
x2 = tf.Variable(2, name='x2')

total = tf.add(x1, x2, name='total')  # total = x1 + x2
init = tf.global_variables_initializer()

In [8]:
# create a session
sess = tf.Session()
sess.run(init)  # initialize variables
print(sess.run(total))
sess.close()

3


![](./screenshot/img1.1-4.png)

## tf.assign

In [9]:
# define graph
x1 = tf.Variable(1, name='x1')
x2 = tf.constant(2, name='x2')

total = tf.add(x1, x2, name='total')  # total = x1 + x2
assign_x1 = tf.assign(x1, total)
init = tf.global_variables_initializer()

In [12]:
# create a session
sess = tf.Session()
sess.run(init)
for _ in range(3):
    print('x1 =', sess.run(x1))
    print('x2 =', sess.run(x2))
    print('total =', sess.run(total))
    print('---')
    sess.run(assign_x1)
    
sess.close()

x1 = 1
x2 = 2
total = 3
---
x1 = 3
x2 = 2
total = 5
---
x1 = 5
x2 = 2
total = 7
---


![](./screenshot/img1.1-5.png)

## 建立自定義的 Graph
1. 在 default graph 操作

In [2]:
a = tf.Variable(1.0, dtype=tf.float32, name='a')
b = tf.Variable(3.0, dtype=tf.float32, name='b')
c = tf.Variable(5.0, dtype=tf.float32, name='c')
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
print(sess.run(a+b+c))
sess.close()

9.0


2. 在自己的區塊自定義Graph

In [3]:
my_graph = tf.Graph()  # create a graph object

with my_graph.as_default():

    d = tf.Variable(2.0, dtype=tf.float32, name='d')
    e = tf.Variable(4.0, dtype=tf.float32, name='e')
    f = tf.Variable(6.0, dtype=tf.float32, name='f')
    init = tf.global_variables_initializer()

sess = tf.Session(graph=my_graph)  # pass a specific graph
sess.run(init)
print(sess.run(d+e+f))
sess.close()

12.0


In [7]:
pprint(tf.global_variables())

[<tf.Variable 'a:0' shape=() dtype=float32_ref>,
 <tf.Variable 'b:0' shape=() dtype=float32_ref>,
 <tf.Variable 'c:0' shape=() dtype=float32_ref>]


In [8]:
with my_graph.as_default():
    pprint(tf.global_variables())

[<tf.Variable 'd:0' shape=() dtype=float32_ref>,
 <tf.Variable 'e:0' shape=() dtype=float32_ref>,
 <tf.Variable 'f:0' shape=() dtype=float32_ref>]
